In [ ]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

# Fetch historical stock data for Apple Inc. (AAPL)
ticker = "AAPL"
data = yf.download(ticker, start="2022-01-01", end="2023-01-01")
closing_prices = data['Close'].values

In [ ]:
# Kalman Filter implementation
n_timesteps = len(closing_prices)
dt = 1.0  # time step
A = np.array([[1, dt], [0, 1]])
H = np.array([[1, 0]])
Q = np.array([[1, 0], [0, 1]])  # process noise covariance
R = np.array([[10]])  # measurement noise covariance
x = np.zeros((2, n_timesteps))  # state vector [price, drift]
P = np.zeros((2, 2, n_timesteps))  # error covariance
filtered_price = np.zeros(n_timesteps)  # filtered prices

In [ ]:
# Initial estimates
x[:, 0] = [closing_prices[0], 0]  # initial state estimate
P[:, :, 0] = np.eye(2) * 1000  # initial error covariance

for t in range(1, n_timesteps):
    # Prediction
    x[:, t] = A @ x[:, t-1]
    P[:, :, t] = A @ P[:, :, t-1] @ A.T + Q
    
    # Update
    K = P[:, :, t] @ H.T @ np.linalg.inv(H @ P[:, :, t] @ H.T + R)
    x[:, t] = x[:, t] + K @ (closing_prices[t] - H @ x[:, t])
    P[:, :, t] = (np.eye(2) - K @ H) @ P[:, :, t]
    
    # Store the filtered price
    filtered_price[t] = x[0, t]

# Plotting results
plt.figure(figsize=(14, 7))
plt.plot(closing_prices, label='Observed Prices')
plt.plot(filtered_price, label='Filtered Prices', linestyle='dashed')
plt.title(f'{ticker} Stock Prices: Observed vs. Filtered')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()